In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

<a id="1"></a> <br>
# 1. Import Necessary Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score , StratifiedKFold

<a id="1"></a> <br>
# 2. Import Dataset

In [ ]:
df_train = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv')
df_test = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/test.csv')
df_items = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/items.csv')
df_shops = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/shops.csv')
df_item_categories = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/item_categories.csv')
df_sub = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/sample_submission.csv')

<a id="1"></a> <br>
# 3. Explore Data

In [ ]:
df_train

In [ ]:
df_train.info()

In [ ]:
df_train.isnull().sum()

In [ ]:
df_test.isnull().sum()

We dont have any missing value on train & test data

<a id="1"></a> <br>
# 4. Data Preprocessing

**** Find Outliers****

In [ ]:
fig, axes = plt.subplots(nrows=1,ncols=2,figsize=(18,6))
# Find outliers in the columns df_train.'item_cnt_day',df_train.'item_price'
cols = ['item_cnt_day','item_price']
for i in range(len(cols)):
    axes[i].boxplot(df_train[cols[i]])
    axes[i].set_xlabel(cols[i])
    axes[i].set_ylabel("Count")
       

**** Remove observed outliers****

In [ ]:
# Remove Outliers
outlier1 = df_train[df_train['item_cnt_day']>2000].index[0]
outlier2 = df_train[df_train['item_price']>300000].index[0]
df_train.drop([outlier1,outlier2],axis=0, inplace=True)

# Reset index
df_train.reset_index(inplace=True, drop=True)

****  Find Anomalies****

In [ ]:
# Find anomalies in the columns df_train.'item_cnt_day',df_train.'item_price'
cols = ['item_cnt_day','item_price']
fig, ax = plt.subplots(ncols = len(cols), figsize = (18,6), sharex = True)
fig.subplots_adjust(wspace=0.5)

for i in range(len(cols)):
  ax[i].plot(df_train[cols[i]])
  ax[i].set_xlabel(cols[i])
  ax[i].set_ylabel("Count")



Observations:

Column 'item_cnt_day' has some negative values which could be items that were 'returned'. Since we will be considering monthly counts, we will leave the negative values in there so that we get a correct count of items sold when we aggregrate by month



<a id="1"></a> <br>
# 5. Feature Engineering

In [ ]:
# Get maximum number in column 'date_block_num' representing the last month in data
df_train_max_month = df_train.date_block_num.max()

# Add column 'date_block_num' to the test dataset. The values would be sales_train_max_month + 1 representing next month
df_test['date_block_num'] = df_train_max_month + 1

# Create df_data table to concat modified df_train and df_test
df_data = pd.concat([df_train,df_test]).reset_index(drop=True)

# Create monthly sales data by aggregating on column 'item_cnt_day' and then rename column to 'item_cnt_month'
monthly_sales = df_data.groupby(by = ['date_block_num','shop_id','item_id'], as_index=False).agg({'item_cnt_day':'sum'})

monthly_sales = monthly_sales.rename(columns={'item_cnt_day':'item_cnt_month'})

<a id="1"></a> <br>
# 6. Data Preparation

In [ ]:
# The 'date_block_num' must be split as contiguos months of training data and validation data
split_ratio = 0.8
train_valid_split = np.floor(df_train_max_month * split_ratio)
train_data = monthly_sales[monthly_sales['date_block_num'] <= train_valid_split]
valid_data = monthly_sales[(monthly_sales['date_block_num'] > train_valid_split) & (monthly_sales['date_block_num'] < df_train_max_month + 1)]

# Test data will be the 'date_block_num' is sales_train_max_month+1
test_data = monthly_sales[monthly_sales['date_block_num'] == df_train_max_month+1]

In [ ]:
# Create X and y variables for train, validation and test sets
X_train = train_data.drop('item_cnt_month',axis=1)
y_train = train_data['item_cnt_month']

X_valid = valid_data.drop('item_cnt_month',axis=1)
y_valid = valid_data['item_cnt_month']

X_test = test_data.drop('item_cnt_month',axis=1)
y_test = test_data['item_cnt_month']

<a id="1"></a> <br>
# 7. Modeling

For the modeling part we will compare 3 known Linear algorithms:

1: LinearRegression

2: Ridge

3: Lasso

In [ ]:
# Regression Algorithms
pipeline_LinearRegression = Pipeline([('LinearRegression_Regression',LinearRegression())])
pipeline_Ridge = Pipeline([('Ridge_Regression',Ridge(alpha=0.1))])
pipeline_Lasso = Pipeline([('Lasso_Regression',Lasso(alpha=0.01))])

pipelines = [pipeline_LinearRegression,pipeline_Ridge,pipeline_Lasso]
pipe_dict = {0:'LinearRegression',1:'Ridge',2:'Lasso'}

In [ ]:
# Fit Model
for pipe in pipelines:
    pipe.fit(X_train,y_train)

In [ ]:
# Use model to Predict labels for validation and test sets 
for i,model in enumerate(pipelines):
    train_pred =  model.predict(X_train)
    valid_pred = model.predict(X_valid)
    test_pred = model.predict(X_test)
    # Error metrics
    print('{} Root Mean Square Train Data = {}'.format(pipe_dict[i],np.sqrt(mean_squared_error(y_train,train_pred))))
    print('{} Root Mean Square Validation Data = {}\n'.format(pipe_dict[i],np.sqrt(mean_squared_error(y_valid,valid_pred)))) 

In [ ]:
# Error metrics
print('Root Mean Square Train Data = {}'.format(np.sqrt(mean_squared_error(y_train,train_pred))))
print('Root Mean Square Validation Data = {}'.format(np.sqrt(mean_squared_error(y_valid,valid_pred)))) 

Submit Predictions

In [ ]:
submission = pd.DataFrame(df_test['ID'])
submission['item_cnt_month'] = pipeline_LinearRegression.predict(X_test)
submission.to_csv('submission.csv',index=False)